In [ ]:
# importing matplotlib
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
# importing everthing not sqlalchemy or matplotlib
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

## Reflecting Tables

In [ ]:
# reflecting
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
# getting names of tables
Base.classes.keys()

In [ ]:
# saving references
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
session = Session(engine)

# Exploring Precipitation Analysis

In [ ]:
# lets look at what tables look like
measurement_f = session.query(Measurement).first()
measurement_f.__dict__

In [ ]:
station_f = session.query(Station).first()
station_f.__dict__

In [ ]:
# i am looking to find the earliest date
early_days = session.query(Measurement.date).order_by(func.strftime('%Y-%m-%d', Measurement.date)).all()

In [ ]:
earliest_date = early_days[0][0]
earliest_date

## i want the last 12 months of precipitation data and plot it

In [ ]:
# this is the last day in the dataset
latest_date = early_days[-1][0]
latest_date

In [ ]:
# finding a year ago
a_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(a_year_ago)

In [ ]:
# i want to see a list of stations
# station_list = session.query(Measurement.station).distinct().all()
# my_station = station_list[5][0]
# my_station

In [ ]:
# i am going to run my query then pop it into a data frame
state = session.query(Measurement.date, Measurement.prcp).\
    filter(func.strftime('%Y-%m-%d', Measurement.date) > a_year_ago).statement

rain = pd.read_sql_query(state, session.bind)

In [ ]:
rain.date = pd.to_datetime(rain.date, format='%Y-%m-%d')

In [ ]:
rain.set_index('date', inplace=True)
rain.head()

In [ ]:
# sorting by date
rain.sort_index(inplace=True)

In [ ]:
# Plotting chart
rain.plot(figsize=(10,5), title='Precipitation')
plt.xlabel('Date')
plt.ylabel('Inches')
plt.show()

In [ ]:
# summary statisics
rain.describe()

# Exploritory Station Analysis

In [ ]:
# how many stations are there
num_stations = session.query(func.count(Station.station)).all()

In [ ]:
print(f'There are {num_stations[0][0]} stations in Hawaii.')

In [ ]:
# which is the most active station
session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).all()

In [ ]:
# ok that looks good, now I need to order by and desc them
stations_desc = session.query(Measurement.station, func.count(Measurement.tobs)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.tobs).desc()).all()

In [ ]:
# station one is the most active station
station_one = stations_desc[0][0]

In [ ]:
# going for the lowest temp
lowest_temp = session.query(func.min(Measurement.tobs)).filter(Measurement.station == station_one).all()[0][0]
lowest_temp

In [ ]:
# going to highest temp
highest_temp = session.query(func.max(Measurement.tobs)).filter(Measurement.station == station_one).all()[0][0]
highest_temp

In [ ]:
# now I want the avg temp at station one
avg_temp = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == station_one).all()[0][0]
avg_temp

## plotting histogram of frequency of temperatures

In [ ]:
# query for temperture over last year
temp_list = session.query(Measurement.tobs).filter(Measurement.station == station_one).\
    filter(func.strftime('%Y-%m-%d', Measurement.date) > a_year_ago).all()

In [ ]:
# so I ran into a problem where my query was a list of tuples so i had to remove them from the tuples so i used 
# list comprehension I got the spring board idea from
# https://stackoverflow.com/questions/46152666/how-to-remove-a-tuple-from-the-list

new_temp_list = [x[0] for x in temp_list]

In [ ]:
# plot of Frequency of Temperatures
plt.figure(figsize=(10,6))
plt.hist(new_temp_list, bins=12)

plt.title('Frequency of Temperatures')
plt.xlabel('Tempertures')
plt.ylabel('Frequency')

plt.show()

In [ ]:
session.close()